In [29]:
import pandas as pd
import openai
import requests
import json
from tqdm import tqdm
from dotenv import load_dotenv
import os



In [30]:
# Load the CSV file with extracted grant information
input_csv_path = '/Users/miguelemb/Downloads/Projects/Grant scrapper/data/scraped_data/scraped_all_grants.csv'


In [31]:
# Limit the number of grants for testing

grants_df = pd.read_csv(input_csv_path).head(10)


In [32]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
5,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/26/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
6,"=HYPERLINK(""https://www.grants.gov/search-resu...","PHS 2024-2 Omnibus Solicitation of the NIH, CD...",HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
7,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH for...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
8,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH for...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
9,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH and...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...


In [33]:
# Load environment variables from .env file
load_dotenv()

# Set up the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [34]:
# Detailed description of Founder School
founder_school_description = (
    "Founder School is a nonprofit organization focused on empowering underrepresented entrepreneurs across the United States, Caribbean, Central America, and South America. "
    "We provide resources, mentorship, and funding opportunities for individuals from marginalized communities who want to start or scale their businesses. "
    "Our primary focus is on promoting inclusive innovation and economic empowerment by providing access to networks, business education, technology training, and support. "
    "Founder School helps founders develop sustainable business models, refine their skills, and secure funding, particularly in underserved regions where traditional venture capital is less accessible. "
    "We represent Black, Latino, women, and other underrepresented entrepreneurs, with a focus on technology companies, especially in crypto, AI, and related fields. "
    "Our mission is to foster a supportive community where aspiring entrepreneurs can learn, connect, and grow, ultimately transforming their ideas into successful ventures."
)


In [35]:
# Define the columns that we will be adding to the DataFrame
columns = ['Apply Decision', 'Reason for Decision', 'Funding Amount','Additional Information URL']

In [36]:
# Ensure these columns are present in the original DataFrame
for col in columns:
    if col not in grants_df.columns:
        grants_df[col] = None


In [37]:
# Initialize a counter for API usage
api_usage_count = 0

In [38]:
grants_df.head()

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text,Apply Decision,Reason for Decision,Funding Amount,Additional Information URL
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None


In [39]:

# Function to analyze grants using GPT-4o-mini
def analyze_grant(grant_info):
    # Create a prompt based on grant information
    prompt = (
        f"Given the following grant information, determine if it is a good fit for Founder School. "
        f"Founder School is described as follows: {founder_school_description}. "
        f"Provide a summary analysis of the grant's fit for Founder School in the following format: "
        f"'Decision: [Apply/Not Apply/Maybe] - Reason: [Reason for the decision] - Funding Amount: [Amount, if available] - Additional Information URL: [URL, if available]'. "
        f"Only provide an answer if you have all the necessary information to make a confident decision. If unsure, say 'Not Sure'."
        f"\nGrant Information:\n{grant_info}"
    )
    
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json=data
    )
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()

In [40]:
# Loop through each grant and apply GPT analysis with a progress bar
for idx in tqdm(range(len(grants_df)), desc="Analyzing Grants"):
    grant_info = grants_df.iloc[idx].to_dict()
    gpt_analysis = analyze_grant(grant_info)
    
    # Increment API usage count
    api_usage_count += 1
    
    # Split GPT analysis into decision, reason, funding amount, and additional information URL
    if "Decision:" in gpt_analysis and "Reason:" in gpt_analysis and "Funding Amount:" in gpt_analysis and "Additional Information URL:" in gpt_analysis:
        try:
            decision_part = gpt_analysis.split("Decision:")[1].split("- Reason:")[0].strip()
            reason_part = gpt_analysis.split("Reason:")[1].split("- Funding Amount:")[0].strip()
            funding_amount_part = gpt_analysis.split("Funding Amount:")[1].split("- Additional Information URL:")[0].strip()
            additional_info_url_part = gpt_analysis.split("Additional Information URL:")[1].strip()
            grants_df.at[idx, 'Apply Decision'] = decision_part
            grants_df.at[idx, 'Reason for Decision'] = reason_part
            grants_df.at[idx, 'Funding Amount'] = funding_amount_part
            grants_df.at[idx, 'Additional Information URL'] = additional_info_url_part
        except IndexError:
            grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"
            grants_df.at[idx, 'Additional Information URL'] = "Error parsing GPT response"
    else:
        grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"
        grants_df.at[idx, 'Additional Information URL'] = "Error parsing GPT response"

# Print the total API usage
print(f"Total API calls made: {api_usage_count}")


Analyzing Grants: 100%|██████████| 10/10 [00:18<00:00,  1.88s/it]

Total API calls made: 10


In [41]:
# Add a column to check if all columns were read before providing an answer
grants_df['All Columns Read'] = grants_df.apply(lambda row: 'Yes' if all(pd.notna(row)) else 'No', axis=1)

In [42]:
# Save the analyzed data to a new CSV file
grants_df.to_csv("/Users/miguelemb/Downloads/Projects/Grant scrapper/data/agent_anlysis/gpt_analysis_results.csv", index=False)

In [43]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text,Apply Decision,Reason for Decision,Funding Amount,Additional Information URL,All Columns Read
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically aimed at for-profit ...,"$10,000,000 (total program funding); Award Cei...",[NSF Grant Opportunity](https://www.grants.gov...,Yes
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted towards ins...,"Estimated Total Program Funding: $31,500,000; ...",[NSF Publication 23-588](https://www.grants.go...,Yes
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted at small bu...,Not specified,[https://grants.nih.gov/grants/guide/pa-files/...,Yes
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted towards sma...,Not specified,https://www.grants.gov/search-results-detail/3...,Yes
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant focuses on supporting small business...,Not specified,[Link to Grant](https://www.grants.gov/search-...,Yes
5,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/26/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically focused on small bus...,Not specified,[Link to Grant](https://grants.nih.gov/grants/...,Yes
6,"=HYPERLINK(""https://www.grants.gov/search-resu...","PHS 2024-2 Omnibus Solicitation of the NIH, CD...",HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted at small bu...,Not specified,[Grant URL](https://www.grants.gov/search-resu...,Yes
7,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH for...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically for small businesses...,Not specified,https://grants.nih.gov/grants/guide/pa-files/P...,Yes
8,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH for...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically for small businesses...,Not specified,[Link](https://grants.nih.gov/grants/guide/pa-...,Yes
9,"=HYPERLINK(""https://www.grants.gov/search-resu...",PHS 2024-2 Omnibus Solicitation of the NIH and...,HHS-NIH11,posted,07/03/2024,04/05/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant opportunity is specifically for smal...,Not specified,[Link](https://www.grants.gov/search-results-d...,Yes
